In [1]:
from data.connect import autoconnect_db
from data.model import Collection
from data.req_nina import *

from datetime import datetime
import requests

In [2]:
engine, session = autoconnect_db()

In [9]:
def print_collection(collection: Collection):
    print(f"name: {collection.name}")
    print(f"url_items: {collection.url_items}") 
    print(f"url_collection: {collection.url_collection}")
    print(f"entries: {collection.entries}")
    # print(f"last_updated: {collection.last_updated}")
    print(f"dataset_id: {collection.dataset_id}")

In [10]:
# get the first collection
collection = session.query(Collection).first()
print_collection(collection)

name: Perspektive Wohnen
url_items: https://api.hamburg.de/datasets/v1/perspektive_wohnen/collections/perspektive_wohnen_bestehend/items?f=json
url_collection: https://api.hamburg.de/datasets/v1/perspektive_wohnen/collections/perspektive_wohnen_bestehend
entries: 15
last_updated: None
dataset_id: 1


In [36]:
def request_items(collection: Collection, verbose=True):
    """
    Takes in a Collection object from the database and requests the dataset items from the API.
    Returns the GeoJSON response from the API or None if the request failed.
    """

    url_items = collection.url_items
    entries = collection.entries
    last_updated = collection.last_updated

    last_updated = datetime(2024, 1, 23, 13, 30)

    # only request items if they were updated since the last request
    str_modified_since = last_updated.strftime("%a, %d %b %Y %H:%M:%S GMT")

    # the header we send with our requests
    headers = {
        "If-Modified-Since": str_modified_since
    }

    print(headers)

    # also add the limit parameter to the url
    # it controls how many items are returned
    # TODO: if the number of items is large, make multiple smaller requests

    url_items = url_items + f"&limit={entries}"
    response = requests.get(url_items, headers=headers)

    print(response.text)

    if response.status_code == 304:
        if verbose: print(f"Items not modified since last request ({str_modified_since})")
        return None

    if response.status_code == 200:

        response_json = response.json()
        if verbose: print(f"{response_json["numberMatched"]} items returned from {response_json["numberReturned"]}")

        return response_json
    
    return None

In [37]:
it = request_items(collection, verbose=True)

{'If-Modified-Since': 'Tue, 23 Jan 2024 13:30:00 GMT'}
{"type":"FeatureCollection","numberReturned":15,"numberMatched":15,"timeStamp":"2024-01-23T13:45:03Z","features":[{"type":"Feature","id":13224,"geometry":{"type":"Point","coordinates":[9.772798635523914,53.58288908024231]},"properties":{"bezeichnung":"Suurheid","bezirk":"Altona","stadtteil":"Rissen","platzzahl":300,"platzzahl_hinweis":"Bei den Platzzahlen handelt es sich um die Normalkapazität. Der tatsächliche Belegungsstand kann abweichen, z.B. aufgrund noch durchzuführender baulicher Maßnahmen.","bemerkung":". Für Standorte der Perspektive Wohnen finden Sie hier weitere Informationen:","bemerkung_link":"http://www.hamburg.de/pressearchiv-fhh/4612224/2015-10-06-bsw-fluechtlingsunterkuenfte-perspektive-wohnen/","inbetriebnahme":"2018"}},{"type":"Feature","id":13234,"geometry":{"type":"Point","coordinates":[9.901888774772415,53.619390265137596]},"properties":{"bezeichnung":"Oliver-Lißy-Strasse","bezirk":"Eimsbüttel","stadtteil":"Ei

In [ ]:
session.close()

In [2]:
# get data from the nina api
base_url = "https://warnung.bund.de/api31/"
service = "dashboard/"

# amtlicher regionalschlüssel: https://www.penultima.de/ars/
ars = "033600000000"  # ars for hamburg

# get the data
url = f"{base_url}{service}{ars}.json"
print(url)
response = requests.get(url)

print(response)

https://warnung.bund.de/api31/dashboard/033600000000.json
<Response [200]>


In [8]:
def display_alert(alert):
    payload = alert["payload"]
    id = payload["id"]
    hash = payload["hash"]
    data = payload["data"]

    headline_de = data["headline"]
    provider = data["provider"]
    severity = data["severity"]
    urgency = data["urgency"]
    msgType = data["msgType"]
    area = data["area"]
    valid = data["valid"]

    headline_en = alert["i18nTitle"]["en"]

    time_sent = alert["sent"]
    time_onset = alert["onset"]
    time_expires = alert["expires"]
    time_effective = alert["effective"]

    print(f"ID: {id}\nHash: {hash}\nHeadline DE: {headline_de}\nProvider: {provider}\nSeverity: {severity}\nUrgency: {urgency}\nMessage Type: {msgType}\nArea: {area}\nValid: {valid}\nHeadline EN: {headline_en}\nTime Sent: {time_sent}\nTime Onset: {time_onset}\nTime Expires: {time_expires}\nTime Effective: {time_effective}\n{"-"*50}")

In [14]:
# aktuelle Warnmeldungen abrufen nach Gebietscode
response = requests.get(base_url + "/dashboard/" + ars + ".json") # TODO: hier pruefen, ob Abruf erfolgreich war

# wenn Abruf erfolgreich war, erhalten wir ein JSON
warnungen = response.json()

# iteriere über alle Warnmeldungen
for warnung in warnungen:
    # Der Dashboard-Abruf enthält nur eine Kurzform der Warnmeldung
    # Deshalb rufen wir hier die Details ab:
    id = warnung["payload"]["id"]
    warning_details = requests.get(base_url + "/warnings/" + id + ".json").json() # TODO: Fehlerbehandlung ergaenzen
    # headline und description Felder aus dem JSON fuer die Ausgabe zusammensetzen
    meldungs_text = warning_details["info"][0]["headline"] + ": " + warning_details["info"][0]["description"]
    print("- " + meldungs_text)

- Hochwasserinformation zur Abflusslage für das Flussgebiet Aller: Es liegt eine Warninformation vom 08.02.2024 16:27 Uhr vor, die auf www.pegelonline.nlwkn.niedersachsen.de zu finden ist.


Example Nina response:
```json
[
  {
    "id": "lhp.LHP.NI.ni_NI_15408",
    "payload": {
      "version": 5,
      "type": "ALERT",
      "id": "lhp.LHP.NI.ni_NI_15408",
      "hash": "392f2a833cc1045070a4d9ab24701a9731c897e27769565b0cb69080a5680d76",
      "data": {
        "headline": "Hochwasserinformation zur Abflusslage für das Flussgebiet Aller",
        "provider": "LHP",
        "severity": "Moderate",
        "urgency": "Immediate",
        "msgType": "Alert",
        "area": {
          "type": "ZGEM",
          "data": "110+1,115+1,166+9,319+1,322+3,327+3,334+1,381,387,391,393+1,396+17,415+1,468+8,482+1,485,491+2,528,530+4,536+26,564+2,568+18,588+3,593+12,609,612,775,780,928"
        },
        "valid": true
      }
    },
    "i18nTitle": {
      "de": "Hochwasserinformation zur Abflusslage für das Flussgebiet Aller"
    },
    "sent": "2024-02-08T16:33:01+01:00",
    "onset": "2024-02-08T16:27:00+01:00",
    "effective": "2024-02-08T16:27:00+01:00"
  }
]
```

You can then request further information from:
https://warnung.bund.de/api31/warnings/lhp.LHP.NI.ni_NI_15408.json
```json
{
  "identifier": "lhp.LHP.NI.ni_NI_15408",
  "sender": "HWVZ@nlwkn.niedersachsen.de",
  "sent": "2024-02-08T16:33:01+01:00",
  "status": "Actual",
  "msgType": "Alert",
  "scope": "Public",
  "code": [
    "DVN:5"
  ],
  "info": [
    {
      "language": "de-DE",
      "category": [
        "Met"
      ],
      "event": "Flood",
      "responseType": [
        "None"
      ],
      "urgency": "Immediate",
      "severity": "Moderate",
      "certainty": "Observed",
      "effective": "2024-02-08T16:27:00+01:00",
      "onset": "2024-02-08T16:27:00+01:00",
      "senderName": "Niedersächsischer Landesbetrieb für Wasserwirtschaft, Küsten- und Naturschutz",
      "headline": "Hochwasserinformation zur Abflusslage für das Flussgebiet Aller",
      "description": "Es liegt eine Warninformation vom 08.02.2024 16:27 Uhr vor, die auf www.pegelonline.nlwkn.niedersachsen.de zu finden ist.",
      "web": "https://www.pegelonline.nlwkn.niedersachsen.de/Start",
      "contact": "Niedersächsischer Landesbetrieb für Wasserwirtschaft, Küsten- und Naturschutz",
      "parameter": [
        {
          "valueName": "PHGEM",
          "value": "110+1,115+1,166+9,319+1,322+3,327+3,334+1,381,387,391,393+1,396+17,415+1,468+8,482+1,485,491+2,528,530+4,536+26,564+2,568+18,588+3,593+12,609,612,775,780,928"
        },
        {
          "valueName": "ZGEM",
          "value": "110+1,115+1,166+9,319+1,322+3,327+3,334+1,381,387,391,393+1,396+17,415+1,468+8,482+1,485,491+2,528,530+4,536+26,564+2,568+18,588+3,593+12,609,612,775,780,928"
        }
      ],
      "area": [
        {
          "areaDesc": "Flussgebiet Aller in Niedersachsen",
          "geocode": [
            {
              "valueName": "AreaId",
              "value": "0"
            }
          ]
        }
      ]
    }
  ]
}
```

In [4]:
alerts = get_alerts()
a = create_alert(alerts[0])

https://warnung.bund.de/api31/dashboard/033600000000.json
200


In [14]:
a.time_effective

datetime.datetime(2024, 2, 8, 16, 27, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))

In [4]:
save_alerts()

1 existing alerts
Comparing against time 2024-02-17 21:04:44.314261
alert time 2024-02-08 15:33:01
['392f2a833cc1045070a4d9ab24701a9731c897e27769565b0cb69080a5680d76']
Recieved duplicate alert, skipping... (hash=392f2a833cc1045070a4d9ab24701a9731c897e27769565b0cb69080a5680d76)


[]